In [1]:
# Importing libraries

import pyspark 
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

### Because of the nature of the project and the fact that I'm not paying for Cloud computing I've decided to do this with all of the information. If the speed or price would've been a factor I would've taken a sample of around 20% of the data

In [2]:
# Where is the 20gb data file?

DATA_PATH = r"C:\Users\jtrev\Downloads\all_data.csv"
DATA_TO_SAMPLE_PATH = r"C:\Users\jtrev\Desktop\wizeline_sample.csv"

In [3]:
#Creating spark session
spark = SparkSession.builder.appName("Wizeline DE bootcamp").getOrCreate()

In [4]:
# Reading the data
df = spark.read.option("header",True).format("csv").load(DATA_PATH)

## Basic exploration with the complete dataset

In [5]:
# Table dimensions
print("Table dimensions: " + str(df.count()) + " filas y "  + str(len(df.columns)) + " columnas")

Table dimensions: 62530715 filas y 15 columnas


In [6]:
# Duplicated records?
total_rows = df.count()
total_unique_rows = df.dropDuplicates().count()
print("Total rows: " + str(total_rows))
print("Total unique rows: " + str(total_unique_rows))
if total_rows > total_unique_rows:
    print('Data has duplicates')

Total rows: 62530715
Total unique rows: 60753811
Data has duplicates


In [7]:
# Column names and how many distinct values each one has
names = df.schema.names
for name in names:
    print(str(name) + " contiene " + str(df.select(name).distinct().count()) + " valores únicos")

producto contiene 1107 valores únicos
presentacion contiene 8244 valores únicos
marca contiene 2079 valores únicos
categoria contiene 42 valores únicos
catalogo contiene 13 valores únicos
precio contiene 133307 valores únicos
fechaRegistro contiene 7487339 valores únicos
cadenaComercial contiene 706 valores únicos
giro contiene 22 valores únicos
nombreComercial contiene 3895 valores únicos
direccion contiene 6526 valores únicos
estado contiene 36 valores únicos
municipio contiene 209 valores únicos
latitud contiene 3564 valores únicos
longitud contiene 3562 valores únicos


In [8]:
# Why are there 36 states? If I am correctly recalling my geography classes there should be only 32
# Probably a problem with accent marks or misspellings
df.select('estado').distinct().collect()

[Row(estado='QUINTANA ROO'),
 Row(estado='NUEVO LEÓN'),
 Row(estado='SINALOA'),
 Row(estado='TABASCO'),
 Row(estado='BAJA CALIFORNIA'),
 Row(estado='TLAXCALA'),
 Row(estado='COAHUILA DE ZARAGOZA'),
 Row(estado=None),
 Row(estado=' ESQ. SUR 125"'),
 Row(estado='CHIAPAS'),
 Row(estado=' COL. EDUARDO GUERRA'),
 Row(estado='VERACRUZ DE IGNACIO DE LA LLAVE'),
 Row(estado='SONORA'),
 Row(estado='YUCATÁN'),
 Row(estado='MICHOACÁN DE OCAMPO'),
 Row(estado='DURANGO'),
 Row(estado='GUERRERO'),
 Row(estado='NAYARIT'),
 Row(estado='CHIHUAHUA'),
 Row(estado='DISTRITO FEDERAL'),
 Row(estado='estado'),
 Row(estado='HIDALGO'),
 Row(estado='ZACATECAS'),
 Row(estado='GUANAJUATO'),
 Row(estado='SAN LUIS POTOSÍ'),
 Row(estado='TAMAULIPAS'),
 Row(estado='MORELOS'),
 Row(estado='AGUASCALIENTES'),
 Row(estado='OAXACA'),
 Row(estado='PUEBLA'),
 Row(estado='BAJA CALIFORNIA SUR'),
 Row(estado='JALISCO'),
 Row(estado='CAMPECHE'),
 Row(estado='QUERÉTARO'),
 Row(estado='COLIMA'),
 Row(estado='MÉXICO')]

In [9]:
# The 4 no-states have been identified, since there is no way to convert this ones to real states I will just ignore them
# in case an analysis with "estado" needs to be done:
estados_to_ignore = [' ESQ. SUR 125"',
                     None,
                     'estado',
                     ' COL. EDUARDO GUERRA',
                     ' ESQ. SUR 125""']


In [10]:
# Since I don't intend to do any analysis relating to geographic position I'm gonna drop direccion, latitud and longitud
columns_to_drop = ['direccion', 'latitud', 'longitud']
df = df.drop(*columns_to_drop)

## Domain specific data exploration
 What is the meaning of each column? </br>
 What is the hierarchy between columns?</br>
 What does each row represent? 


In [11]:
# I will take a bigger and random sample to what was given and do a quick exploration in excel to validate some assumptions
# Will comment since its not required for execution
# df.sample(.01).toPandas().to_csv(DATA_TO_SAMPLE_PATH, index = False)

# Reading the sampled file into pyspark for testing, will comment this at the end and run the notebook with the complete file
# df = spark.read.option("header",True).format("csv").load(DATA_TO_SAMPLE_PATH")

### Let's start by defining the columns

 We are looking at a table of specific products that Profeco keeps track of, each product is linked to a store 'nombreComercial' </br>
 Each SKU is composed of the columns producto-presentacion-marca</br>
 Each 'producto' is linked to a category 'categoria' which is linked to a catalog 'catalogo'# Each SKU has a price at a given 'fechaRegisto' </br>
 General Hierarchy is: cadenaComercial > nombreComercial > producto-marca > presentacion > fechaRegistro > precio</br>
 SKU hierarchy is: catalogo > categoria >  producto-marca > presentacion</br>

 producto: what kind of product it is</br>
 presentacion: in what presentation does this product come</br>
 marca: product brand</br>
 categoria: product category</br>
 catalogo: the bigger category of each category</br>
 precio: product price at fechaRegistro</br>
 fechaRegistro: At what date was this product with this price registered in the database?</br>
 cadenaComercial: The commercial chain of each store</br>
 giro: Category that Profeco uses to designate each store</br>
 nombreComercial: Store that owns the actual product</br>
 estado: Mexican state where the store is located</br>
 municipio: Mexican city where the store is located</br>

## How many commercial chains are monitored, and therefore, included in this database?


In [12]:
# Assuming that all values are correct and there are not None values
print(df.select('cadenaComercial').distinct().count())

706


## What are the top 10 monitored products by State?

In [13]:
# Assuming this question is referring to: the 10 monitored product that have more records in the database by state
df_products_pandas = df.groupby(['estado', 'producto']).count().toPandas()

In [14]:
df_products_pandas = df_products_pandas.sort_values('estado')

In [15]:
estados = df_products_pandas['estado'].drop_duplicates()
for estado in estados:
    if estado in estados_to_ignore:
        continue
    else:
        print(df_products_pandas.loc[df_products_pandas['estado'] == estado].sort_values('count', ascending = False).head(10))

               estado                 producto  count
8910   AGUASCALIENTES                      FUD  12005
25572  AGUASCALIENTES        DETERGENTE P/ROPA  10188
5849   AGUASCALIENTES  LECHE ULTRAPASTEURIZADA   9824
12724  AGUASCALIENTES                  SHAMPOO   9654
26368  AGUASCALIENTES                 REFRESCO   9481
18237  AGUASCALIENTES              DESODORANTE   8859
4188   AGUASCALIENTES         JABON DE TOCADOR   8517
960    AGUASCALIENTES           CHILES EN LATA   7946
25277  AGUASCALIENTES                  YOGHURT   7401
22308  AGUASCALIENTES                 MAYONESA   7173
                estado           producto  count
11182  BAJA CALIFORNIA           REFRESCO  37243
16541  BAJA CALIFORNIA  DETERGENTE P/ROPA  23395
22844  BAJA CALIFORNIA                FUD  19967
6608   BAJA CALIFORNIA            SHAMPOO  19123
16364  BAJA CALIFORNIA   JABON DE TOCADOR  18348
7547   BAJA CALIFORNIA     CHILES EN LATA  16676
21478  BAJA CALIFORNIA           GALLETAS  15873
13737  BAJA CA

       estado                 producto   count
15635  MÉXICO                 REFRESCO  194939
2878   MÉXICO                      FUD  149141
22277  MÉXICO        DETERGENTE P/ROPA  132862
5371   MÉXICO  LECHE ULTRAPASTEURIZADA  116522
25540  MÉXICO         JABON DE TOCADOR   97330
1238   MÉXICO                  YOGHURT   94852
19957  MÉXICO                 MAYONESA   94286
18523  MÉXICO           CHILES EN LATA   92539
25699  MÉXICO                  SHAMPOO   92307
25859  MÉXICO                  CERVEZA   91747
        estado              producto  count
19578  NAYARIT              REFRESCO   8003
26779  NAYARIT             PANTALLAS   7083
14522  NAYARIT                   FUD   6644
30605  NAYARIT             LAVADORAS   6549
23764  NAYARIT     DETERGENTE P/ROPA   5514
14693  NAYARIT  COMPONENTES DE AUDIO   4999
14844  NAYARIT               SHAMPOO   4975
23309  NAYARIT      JABON DE TOCADOR   4680
28405  NAYARIT        REFRIGERADORES   4578
8170   NAYARIT              PLANCHAS   4509

## Which is the commercial chain with the highest number of monitored products?


In [16]:
# Assuming the question is referring to: Which commercial chain has the biggest variety of different products to be monitored
df.groupby('cadenaComercial').agg(fn.countDistinct("producto")).orderBy(fn.desc('count(producto)')).limit(1).show()

+---------------+---------------+
|cadenaComercial|count(producto)|
+---------------+---------------+
|        SORIANA|           1059|
+---------------+---------------+



## Use the data to find an interesting fact.
#### I will answer 2 questions
First: Which are the 10 SKUs with the highest standard deviation per year, taking into account the average price for each year for each one of them </br> </br>
Second: how have their prices varied each year

In [17]:
# Using my previous definition of SKU
df = df.withColumn('SKU', fn.concat(df.producto, df.presentacion, df.marca))
# Creating year column
df = df.withColumn('year',fn.year(fn.to_timestamp('fechaRegistro')))
# Converting precio to number
df = df.withColumn('precio', fn.col('precio').cast(DoubleType()))
# Grouping each SKU by price by year
df_avg_price_per_year_per_sku = df.groupby('SKU', 'year').avg('precio').toPandas()


In [18]:
# Finding which SKU has had a bigger variation of price per year
df_skus_with_high_var = df_avg_price_per_year_per_sku.groupby(['SKU']).std()['avg(precio)'].reset_index()\
                             .sort_values('avg(precio)', ascending = False).dropna()\
                             .head(10)

In [19]:
#Formatting and printing
df_skus_with_high_var['avg(precio)'] = df_skus_with_high_var['avg(precio)'].map('${:,.2f}'.format)
df_skus_with_high_var

,SKU,avg(precio)
6667,PANTALLASLC-70UD1U. 70 PLGS. LED 3D. ENTRADA U...,"$24,546.36"
6769,PANTALLASXBR-65X900A. 65 PLGS. LED. ENTRADA US...,"$14,095.01"
6748,PANTALLASUN 55HU7250F. 55 PLGS. LED. CURVED. C...,"$12,545.74"
6768,PANTALLASXBR-65HX920. COLOR 65 PLGS. PANTALLA ...,"$11,968.41"
6672,PANTALLASLC-80LE844U. COLOR 80 PLGS. PANTALLA ...,"$10,122.36"
6662,PANTALLASLC-60UD27U. 60 PLGS. LED. SMART TVSHA...,"$9,941.10"
6743,PANTALLASUN 55F9000AF. 55 PLGS. LED 3D. C/RECU...,"$9,841.43"
6670,PANTALLASLC-80LE642U. 80 PLGS. LED. SMART TVSH...,"$9,661.28"
6673,PANTALLASLC-80LE857U. 80 PLGS. PANTALLA LED 3D...,"$6,982.14"
6638,PANTALLASKDL-55W800A. 55 PLGS. LED 3D. ENTRADA...,"$6,396.20"


In [20]:
# Mapping them so we can see the price of each of the top 10 skus per year
df_avg_price_per_year_per_sku['avg(precio)'] = df_avg_price_per_year_per_sku['avg(precio)'].map('${:,.2f}'.format)
df_avg_price_per_year_per_sku.loc[df_avg_price_per_year_per_sku['SKU'].isin(df_skus_with_high_var['SKU'])].sort_values('SKU')\
.pivot(index = 'SKU', columns = 'year')

avg(precio)               \
year                                                    2012.0       2013.0   
SKU                                                                           
PANTALLASKDL-55W800A. 55 PLGS. LED 3D. ENTRADA ...         NaN   $35,282.76   
PANTALLASLC-60UD27U. 60 PLGS. LED. SMART TVSHAR...         NaN          NaN   
PANTALLASLC-70UD1U. 70 PLGS. LED 3D. ENTRADA US...         NaN          NaN   
PANTALLASLC-80LE642U. 80 PLGS. LED. SMART TVSHA...         NaN          NaN   
PANTALLASLC-80LE844U. COLOR 80 PLGS. PANTALLA L...  $92,419.02   $78,103.84   
PANTALLASLC-80LE857U. 80 PLGS. PANTALLA LED 3D....         NaN   $77,068.59   
PANTALLASUN 55F9000AF. 55 PLGS. LED 3D. C/RECUA...         NaN   $58,000.18   
PANTALLASUN 55HU7250F. 55 PLGS. LED. CURVED. C/...         NaN          NaN   
PANTALLASXBR-65HX920. COLOR 65 PLGS. PANTALLA L...  $62,664.20   $45,738.31   
PANTALLASXBR-65X900A. 65 PLGS. LED. ENTRADA USB...         NaN  $109,872.87   

                                                                            \
year                                                    2014.0      2015.0   
SKU                                                                          
PANTALLASKDL-55W800A. 55 PLGS. LED 3D. ENTRADA ...  $27,073.13  $22,681.75   
PANTALLASLC-60UD27U. 60 PLGS. LED. SMART TVSHAR...         NaN  $35,940.16   
PANTALLASLC-70UD1U. 70 PLGS. LED 3D. ENTRADA US...  $70,393.00  $35,679.20   
PANTALLASLC-80LE642U. 80 PLGS. LED. SMART TVSHA...         NaN  $53,335.88   
PANTALLASLC-80LE844U. COLOR 80 PLGS. PANTALLA L...         NaN         NaN   
PANTALLASLC-80LE857U. 80 PLGS. PANTALLA LED 3D....  $67,194.36         NaN   
PANTALLASUN 55F9000AF. 55 PLGS. LED 3D. C/RECUA...  $44,082.31         NaN   
PANTALLASUN 55HU7250F. 55 PLGS. LED. CURVED. C/...         NaN  $30,500.36   
PANTALLASXBR-65HX920. COLOR 65 PLGS. PANTALLA L...         NaN         NaN   
PANTALLASXBR-65X900A. 65 PLGS. LED. ENTRADA USB...  $89,939.52         NaN   

                                                                
year                                                    2016.0  
SKU                                                             
PANTALLASKDL-55W800A. 55 PLGS. LED 3D. ENTRADA ...         NaN  
PANTALLASLC-60UD27U. 60 PLGS. LED. SMART TVSHAR...  $49,999.00  
PANTALLASLC-70UD1U. 70 PLGS. LED 3D. ENTRADA US...         NaN  
PANTALLASLC-80LE642U. 80 PLGS. LED. SMART TVSHA...  $66,999.00  
PANTALLASLC-80LE844U. COLOR 80 PLGS. PANTALLA L...         NaN  
PANTALLASLC-80LE857U. 80 PLGS. PANTALLA LED 3D....         NaN  
PANTALLASUN 55F9000AF. 55 PLGS. LED 3D. C/RECUA...         NaN  
PANTALLASUN 55HU7250F. 55 PLGS. LED. CURVED. C/...  $12,758.00  
PANTALLASXBR-65HX920. COLOR 65 PLGS. PANTALLA L...         NaN  
PANTALLASXBR-65X900A. 65 PLGS. LED. ENTRADA USB...         NaN

## What are the lessons learned from this exercise?

Having a data dictionary that describes exactly what does each column means, their hierarchy and what does each row represent would have reduced the time spent doing the analysis by a lot, since that is the part that consumed me the most time, and even then I don't feel 100% percent sure of my definitions

## Can you identify other ways to approach this problem? Explain.

First of all the infrastructure, if this problem could've been solver on a proper infrastructure the results would have been much faster. </br>
The language and frameworks: this was done in python using pyspark, this could've also been done in SQL or python-pandas among other frameworks </br>
The steps in which you should complete this task: I definitely lost a lot of time trying to answer the given questions of the data without first understanding the schema, hierarchy and definitions </br>
Goals: Knowing what would be the specific endgoal of this information would have given me more direction as of how to transform the data, but I understand the questions were ambiguous on purpose </br>
Results could've been cleaner </br>
Everything could've been done in PySpark, I chose to do some things in pandas only due to my familiarity with the library